<a id="1"></a>
# <div style="padding: 20px; color: cyan; margin: 0; font-size: 30px; font-family: Arial; text-align: left; border-radius: 5px; background-color: #000000; border: 2px solid blue;"><b>Machine Learning Model</b></div>

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import tensorflow as tf

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Data </b> Preparation

In [2]:
df = pd.read_csv("../data/processed/processed_data.csv")
df.head(5)

,name,designer,artist,min_players,max_players,min_playtime,max_playtime,playing_time,min_age,complexity_rating,user_ratings,user_comments,type,category,mechanism,year,average_rating
0,Samurai,Reiner Knizia,Franz Vohwinkel,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Area Majority / Influence,1998.0,7.5
1,Samurai,Reiner Knizia,Franz Vohwinkel,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Hand Management,1998.0,7.5
2,Samurai,Reiner Knizia,Franz Vohwinkel,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Hexagon Grid,1998.0,7.5
3,Samurai,Reiner Knizia,Franz Vohwinkel,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Once-Per-Game Abilities,1998.0,7.5
4,Samurai,Reiner Knizia,Franz Vohwinkel,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Set Collection,1998.0,7.5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42369 entries, 0 to 42368
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               42369 non-null  object 
 1   designer           42369 non-null  object 
 2   artist             42369 non-null  object 
 3   min_players        42369 non-null  int64  
 4   max_players        42369 non-null  int64  
 5   min_playtime       42369 non-null  int64  
 6   max_playtime       42369 non-null  int64  
 7   playing_time       42369 non-null  int64  
 8   min_age            42369 non-null  int64  
 9   complexity_rating  42369 non-null  float64
 10  user_ratings       42369 non-null  int64  
 11  user_comments      42369 non-null  int64  
 12  type               42369 non-null  object 
 13  category           42369 non-null  object 
 14  mechanism          42369 non-null  object 
 15  year               42369 non-null  float64
 16  average_rating     423

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Feature </b> engineering

In [6]:
df.drop(columns=['name', 'designer', 'artist', 'year'], inplace=True)
df.head()

,min_players,max_players,min_playtime,max_playtime,playing_time,min_age,complexity_rating,user_ratings,user_comments,type,category,mechanism,average_rating
0,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Area Majority / Influence,7.5
1,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Hand Management,7.5
2,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Hexagon Grid,7.5
3,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Once-Per-Game Abilities,7.5
4,2,4,30,60,45,10,2.47,16000,4000,Strategy,Abstract Strategy,Set Collection,7.5


<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Training </b> model

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Evaluation </b>

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Prediction </b>

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Results </b> analysis

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Conclusion </b> 